## Read KML files

In [55]:
from fastkml import kml
import zipfile

kmzs = ['ICEYE_archive_catalog_2024.kmz', 'ICEYE_archive_catalog_2023.kmz', 'ICEYE_archive_catalog_2022.kmz']
folder_names = []
for kmz in kmzs:
    # read KML content 
    kmz = zipfile.ZipFile(kmz, 'r')  # extract zip file first, then read kmz file inside the extracted folder
    kml_name = kmz.namelist()[0]
    kml_content = kmz.open(kml_name, 'r').read()  # kml content
    
    # create KML object
    k = kml.KML()
    k.from_string(kml_content)
    
    document = list(k.features())
    parent_folder = list(document[0].features())
    folders = list(parent_folder[0].features())    
    subfolder_names = []
    for folder in folders:
        subfolder_names.append(folder.name[14:])
    folder_names.append(subfolder_names)

## Make an Excel file

In [61]:
import os
import pandas as pd
from natsort import natsorted

sheet_names = ['2024', '2023', '2022']
excel_path = 'ICEYE_Archive.xlsx'

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(excel_path, engine='xlsxwriter')

# Get the xlsxwriter workbook.
workbook = writer.book

# Set formats of columns and head 
head_format = workbook.add_format({'align': 'center', 'valign': 'vcenter', 'border': 1, 'bg_color': 'D9D9D9'})
border_format = workbook.add_format({'border': 1})
center_format = workbook.add_format({'align': 'center', 'valign': 'vcenter'})
left_format = workbook.add_format({'align': 'left', 'valign': 'vcenter'})

for idx in range(len(kmzs)):
    df = pd.DataFrame({'No.': list(range(1, len(folder_names[idx])+1)),
                       'Scene ID': folder_names[idx]})
    df['Worker'] = ''  # add an empty column
    df['OilSpill'] = ''
    
    # Get the dimensions of the dataframe.
    (max_row, max_col) = df.shape
    
    # Convert the dataframe to an XlsxWriter Excel object. Turn off the default
    # header and index and skip one row to allow us to insert a user defined header.
    df.to_excel(writer, sheet_name=sheet_names[idx], startrow=0, header=True, index=False)
    
    # Get worksheet objects.
    worksheet = writer.sheets[sheet_names[idx]]    
    worksheet.conditional_format(1, 0, max_row, max_col-1, {'type': 'no_blanks', 'format': border_format})
    worksheet.conditional_format(1, 0, max_row, max_col-1, {'type': 'blanks', 'format': border_format})
    worksheet.conditional_format(1, 2, max_row, 2, {'type': 'no_blanks', 'format': border_format})
    worksheet.conditional_format(0, 0, 0, max_col-1, {'type': 'no_blanks', 'format': head_format})
    worksheet.set_column(0,max_col-1, None, center_format)
    worksheet.set_column(2,2, None, left_format)
    worksheet.autofit()  # autofitting of column widths

# Close the Pandas Excel writer and output the Excel file.
writer.close()